In [1]:
import tengp
import numpy as np
from gpbenchmarks import get_data
from sklearn.metrics import mean_squared_error

In [2]:
def cost_function(*args, **kwargs):
    try:
        return mean_squared_error(*args, **kwargs)
    except ValueError as e:
        return params.fitness_of_invalid


In [3]:
def pdivide(x, y):
    return np.divide(x, y, out=np.copy(x), where=x!=0)

def pow2(x):
    return x**2

def pow3(x):
    return x**3

funset = tengp.FunctionSet()
funset.add(np.add, 2)
funset.add(np.multiply, 2)
funset.add(np.subtract, 2)
funset.add(pdivide, 2)
funset.add(pow2, 1)
funset.add(pow3, 1)

X, y = get_data('nguyenf4', 20, -1, 1)
X = np.c_[np.ones(len(X)), X]

params = tengp.Parameters(2, 1, 1, 100, funset)

In [4]:
builder = tengp.individual.IndividualBuilder(params)

In [23]:
population = [builder.create() for _ in range(5000)]

In [24]:
for individual in population:
    output = individual.transform(X)
    individual.fitness = cost_function(output, y)

/home/jarino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in true_divide
  
/home/jarino/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:116: RuntimeWarning: invalid value encountered in multiply
  current_node.value = current_node.fun(*values)
/home/jarino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:116: RuntimeWarning: invalid value encountered in subtract
  current_node.value = current_node.fun(*values)
/home/jarino/anaconda3/lib/python3.6/site-packages/sklearn/metrics/regression.py:239: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0,
/mnt/c/Users/jarinov-nota

In [25]:
#[print(x.genes, x.fitness) for x in population];

In [26]:
population = [x for x in population if x.fitness != np.inf]

In [27]:
errors = [{} for _ in range(len(population[0].genes))]

for individual in population:
    for index, gene in enumerate(individual.genes):
        if gene in errors[index]:
            errors[index][gene].append(individual.fitness)
        else:
            errors[index][gene] = [individual.fitness]

for index, g_errors in enumerate(errors):
    for gene in g_errors.keys():
        errors[index][gene] = np.min(g_errors[gene])

In [28]:
genes = []

for error in errors:
    gene = min(error.items(), key=lambda x: x[1])[0]
    genes.append(gene)

In [29]:
[print(g, end=',') for g in genes];

0,1,0,2,0,0,1,2,1,1,2,1,2,0,2,4,3,1,5,7,6,3,7,0,0,1,3,2,7,5,0,7,10,4,9,5,5,7,9,1,0,13,5,10,12,1,6,1,4,1,5,2,9,5,0,12,7,1,0,19,1,9,16,5,0,11,4,12,17,2,20,23,3,10,8,5,25,23,3,26,22,2,26,1,0,7,25,1,30,18,5,19,26,0,0,5,0,5,1,5,2,28,2,33,3,2,8,17,3,16,19,4,23,19,3,37,34,2,1,24,3,13,9,5,27,35,0,38,39,3,6,40,3,45,14,5,22,30,4,46,33,4,12,39,1,36,25,5,33,46,0,38,24,2,40,51,4,11,26,0,15,15,3,26,22,3,16,25,0,54,12,4,56,3,3,40,26,3,24,29,1,54,59,2,1,40,3,53,10,1,17,17,3,37,4,4,2,27,5,7,57,5,49,50,5,3,49,0,26,52,4,31,66,3,37,3,4,57,48,0,36,22,3,8,48,1,42,30,4,64,73,2,2,21,2,49,43,3,8,70,1,14,39,3,53,14,1,68,2,1,25,37,4,76,50,0,14,48,4,29,28,4,63,79,2,46,28,3,26,8,0,4,3,1,15,72,3,48,44,3,3,29,1,82,93,3,76,49,0,17,28,3,65,38,3,1,77,1,61,11,58,

In [30]:
individual = params.individual_class(genes, population[0].bounds, params)
print(individual.get_expression())
output = individual.transform(X)
print(cost_function(output, y))

['add(add(add(subtract(x0,x0),x1),pow2(subtract(x0,x0))),pow2(subtract(multiply(x1,add(x0,x1)),pow2(subtract(x0,x0)))))']
0.14433376125516406


In [57]:
def sampling(params):
    builder = tengp.individual.IndividualBuilder(params)
    population = [builder.create() for _ in range(1000)]
    for individual in population:
        output = individual.transform(X)
        individual.fitness = cost_function(output, y)

    population = [x for x in population if x.fitness != np.inf]
    
    errors = [{} for _ in range(len(population[0].genes))]

    for individual in population:
        for index, gene in enumerate(individual.genes):
            if not individual.active_gene(index):
                continue
            if gene in errors[index]:
                errors[index][gene].append(individual.fitness/len(individual.active_nodes))
            else:
                errors[index][gene] = [individual.fitness/len(individual.active_nodes)]

    for index, g_errors in enumerate(errors):
        for gene in g_errors.keys():
            errors[index][gene] = np.mean(g_errors[gene])
    
    genes = []

    for error in errors:
        gene = min(error.items(), key=lambda x: x[1])[0]
        genes.append(gene)
    
    individual = params.individual_class(genes, population[0].bounds, params)
#    print(individual.get_expression())
    output = individual.transform(X)
 #   print(cost_function(output, y))
    fitness = cost_function(output, y)
    print(fitness, min(population, key=lambda x: x.fitness))
    return fitness

In [58]:
results = []                                                                                                                                                                                                                                                                                                                              
for i in range(100):
    res = sampling(params)
    print(i, res)
    results.append(res)

/home/jarino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in true_divide
  
/home/jarino/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
/home/jarino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:116: RuntimeWarning: invalid value encountered in multiply
  current_node.value = current_node.fun(*values)
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:116: RuntimeWarning: invalid value encountered in add
  current_node.value = current_node.fun(*values)
/mnt/c/Users/jarinov-notas/dev/tengp/tengp/individual.py:116: RuntimeWarning: invalid value encountered in subtract
  current_node.value = current_node.fun(*values)


1707765777.4732509 Program, f:0.2830150909042682
0 1707765777.4732509


/home/jarino/anaconda3/lib/python3.6/site-packages/sklearn/metrics/regression.py:239: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0,
/home/jarino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: overflow encountered in power
  


3.4347747345772617 Program, f:0.4076611160984072
1 3.4347747345772617
1.4129193671606725e+50 Program, f:0.26752392881037407
2 1.4129193671606725e+50


/home/jarino/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in square
  """


3.4347747345772617 Program, f:0.26752392881037407
3 3.4347747345772617
52.70313728217059 Program, f:0.132021769160013
4 52.70313728217059
inf Program, f:0.26752392881037407
5 inf
3.4347747345772617 Program, f:0.2784136389052616
6 3.4347747345772617
3.4347747345772617 Program, f:0.2742676935109351
7 3.4347747345772617
8.888832418772349 Program, f:0.137437724574547
8 8.888832418772349
46.08379540676639 Program, f:0.4063265000767804
9 46.08379540676639
2.8290067907268743 Program, f:0.25402891104250636
10 2.8290067907268743
2.8290067907268743 Program, f:0.31441834609848207
11 2.8290067907268743


KeyboardInterrupt: 